In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#export
from exp.nb_02 import *

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Codes" data-toc-modified-id="Codes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Codes</a></span></li><li><span><a href="#Labels" data-toc-modified-id="Labels-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Labels</a></span></li><li><span><a href="#Cropping" data-toc-modified-id="Cropping-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cropping</a></span></li></ul></div>

In [ ]:
print_competition_data()

data/severstal/sample_submission.csv
data/severstal/train.csv
data/severstal/labels
data/severstal/train_images
data/severstal/test_images
data/severstal/.ipynb_checkpoints
data/severstal/codes.txt


For FastAI Segmentation DataLoader we need to build :

1. the codes: the type of pixels in the images

2. the labels: the masks for each image

3. Cropping the images

# Codes

(Optional) Codes are the labels for the corrisponding pixel values in the mask images.

In [ ]:
codes_file_name = "codes.txt"
codes_path = Path(path/codes_file_name)

if not codes_path.is_file():
    classes = np.unique(train.ClassId.values)
    classes = np.insert(classes, 0, 0)
    np.savetxt(path/'codes.txt', classes, fmt='%u')
else:
    classes = np.loadtxt(codes_path, dtype=np.uint8)

classes

array([0, 1, 2, 3, 4], dtype=uint8)

# Labels

Labels are the masks for the training images. 

As a first approach, i tried to create masks only for the images that present a defect. But this will exclude all the good images that can be used to teach to the model to recognize good steel pixels.

Now I created the masks for all the images in the training folder.

**NB: masks must be PNG files and not JPEG because JPEG's compression makes the labels get messed up occasionally** ([source](#https://forums.fast.ai/t/unet-learner-failing-with-a-cuda-device-assert/66747/4))

In [ ]:
labels_dir = path / "labels"

def create_masks(df: pd.DataFrame):
    """Create the mask files under the labels_dir"""
    for img_path in get_selected_imgs_path(df):
        mask = multi_rle_to_mask(img_path)
        im = Image.fromarray(mask)
        im.save(labels_dir/f"{img_path.stem}_P.png")

In [ ]:
# !rm -rf {str(labels_dir)}

In [ ]:
if not labels_dir.is_dir():
    labels_dir.mkdir(parents=True, exist_ok=True)
    create_masks(train_all)

labels_dir.ls()

(#12568) [Path('data/severstal/labels/54fd0b997_P.png'),Path('data/severstal/labels/5f5a2e357_P.png'),Path('data/severstal/labels/3ab4ca44c_P.png'),Path('data/severstal/labels/5b202c6f5_P.png'),Path('data/severstal/labels/026183d85_P.png'),Path('data/severstal/labels/5acf0f8cd_P.png'),Path('data/severstal/labels/79b696401_P.png'),Path('data/severstal/labels/65d8eae69_P.png'),Path('data/severstal/labels/637a0124c_P.png'),Path('data/severstal/labels/f95e24d0a_P.png')...]

With the PNG extension, the multi-label segmentation problem would seem desappear.

In [ ]:
mask_paths = get_image_files(labels_dir)
dotest = False

In [ ]:
# Test if all is working as expected
if dotest:
    n = len(mask_paths)
    idx = get_random_idx(n)
    pixels = set()
    for p in mask_paths[idx]:
        pixels_founded = np.unique(np.array(Image.open(p)))
        pixels = pixels.union(set(pixels_founded))
    pixels

# Cropping

The standard size of the images is `(256, 1600)` and maybe it's because of this that I always go out of memory in the GPU.